In [104]:
import cx_Oracle as cx
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [137]:
#Connect to Autonomous Data Warehouse 
con = cx.connect("ADMIN","yourPass","mltest_high")

In [138]:
query = 'SELECT * from boston_housing'
data_train = pd.read_sql(query, con=con)

query = 'SELECT * from boston_housing_test'
data_test = pd.read_sql(query, con=con)

In [139]:
data_train.head()

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9


In [140]:
data_test.head()

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,None
1,6,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,None
2,8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15,None
3,9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,None
4,10,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,None


In [141]:
######### Building Model with Boosting Method for Regression #######
from sklearn.ensemble import AdaBoostRegressor

X = data_train.iloc[:,1:14] # features
y = data_train.iloc[:,14:15] # target variable

#training
regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X, y) 

#see feature importance
regr.feature_importances_

array([4.74633582e-02, 4.30602322e-04, 5.32814725e-03, 1.97135460e-05,
       1.97097953e-02, 2.44895998e-01, 2.68512864e-02, 1.73748580e-01,
       1.34529983e-02, 4.15055013e-02, 4.81307277e-02, 1.79089420e-02,
       3.60554349e-01])

In [142]:
#model test with test data set from oracle database (data_test df)
data_test.iloc[1:2,1:14] # take one record from data set

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT
1,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222,18.7,394.12,5.21


In [143]:
test_rec = data_test.iloc[1:2,1:14]
testid = data_test.iloc[1:2,0:1] # get test record id for updating result in ADW (database)
id_final = int(test_id["ID"])

In [144]:
#predict value using model
res = regr.predict(test_rec)
pred_medv = res[0]

In [145]:
#update database record on ADW
query = 'update boston_housing_test set medv =:result where id=:testrecid'
cur = con.cursor()
cur.execute(query,[pred_medv,id_final])
con.commit()
con.close()